In [1]:
import pandas as pd
import json
import os
import hana_ml
from hana_ml import ConnectionContext

print(pd.__version__)
print(hana_ml.__version__)

with open(os.path.join(os.getcwd(), './config/cesco-poc-hc-service-key.json')) as f:
    hana_env_c = json.load(f)
    port_c = hana_env_c['port']
    user_c = hana_env_c['user']
    host_c = hana_env_c['host']
    pwd_c = hana_env_c['pwd']

cc = ConnectionContext(address=host_c, port=port_c, user=user_c, password=pwd_c, encrypt=True)
cursor = cc.connection.cursor()
cursor.execute("""SET SCHEMA GEN_AI""")

#Check hana_ml version
print(cc.hana_version())
#Check current schema
print(cc.get_current_schema())

2.2.2
2.21.24072600
4.00.000.00.1721126161 (fa/CE2024.14)
GEN_AI


In [2]:
#get embedding
from gen_ai_hub.proxy.native.openai import embeddings

def get_embedding(input, model="dc872f9eef04c31a") -> str:
    response = embeddings.create(
        deployment_id = model,
        input = input
    )
    return response.data[0].embedding

In [3]:
#get embedding
from gen_ai_hub.proxy.native.openai import embeddings

def run_vector_search_kr(query: str, metric="COSINE_SIMILARITY", k=5):

    if metric == 'L2DISTANCE':
        sort = 'ASC'
        col = 'L2D_SIM'
    elif metric == 'COSINE_SIMILARITY':
        sort = 'DESC'
        col = 'COS_SIM'

    query_vector = get_embedding(query)

    sql = '''SELECT TOP {k} "ProblemDescription","ProblemCategory", "ProblemKeyword",
    "Solution","SolutionDoc", "AdditionalInfo",
    "Code", "{metric}"("VectorProblem", TO_REAL_VECTOR('{qv}'))
    AS "{col}", "CreateDate", "UpdateDate"
    FROM "CESCO_PROBLEMSOLUTIONS22"
    ORDER BY "{col}" {sort}
    '''.format(k=k, metric = metric, qv = query_vector, sort=sort, col=col)

    hdf = cc.sql(sql)
    df_context = hdf.collect()

    return df_context

In [40]:
query = "PDF 파일 열람 오류 해결방법 알려주세요"
df_context = run_vector_search_kr(query=query, metric="COSINE_SIMILARITY", k=5)
df_context

,ProblemDescription,ProblemCategory,ProblemKeyword,Solution,SolutionDoc,AdditionalInfo,Code,COS_SIM,CreateDate,UpdateDate
0,PDF 파일 열람 오류,MS Office 및 기타 프로그램,PDF,1.PDF 파일 형식에 맞는 연결 프로그램을 지정하세요. 연결 프로그램을 사용자가 ...,cesco3_0.pdf,Adobe Acrobat Reader DC 설치 방법 참고Adobe Acroba...,cesco3,0.910578,2024-08-30,2024-08-30
1,"PPT(파워포인트, PowerPoint) 파일 열람 오류",MS Office 및 기타 프로그램,PPT,1.윈도우→설정2.‘앱’ 클릭3.‘기본 앱’ 클릭4.‘파일 형식별 기본 앱 선택’ ...,cesco3_15.pdf,,cesco3,0.875997,2024-08-30,2024-08-30
2,"그룹웨어 메일, 게시판에 첨부된 문서 '열기'클릭 시 열람 오류",그룹웨어,그룹웨어,"1.크롬, 엣지 사이트 접속2.키보드 Ctrl + Shift + Delete 클릭3...",cesco5_11.pdf,,cesco5,0.853127,2024-08-30,2024-08-30
3,'DRM RunDLL / f_piimgr.exe 시스템 오류 해결 방법,보안프로그램,문서보안(DRM),1.윈도우 검색 (윈도우 키 + S) → DRM 응급복구 → 관리자 권한으로 실행2...,cesco8_7.pdf,,cesco8,0.845945,2024-08-30,2024-08-30
4,그룹웨어 게시물에 첨부된 문서 실행 시 빈 화면으로 보이는 문서열람 오류,MS Office 및 기타 프로그램,MS Office,1.문서 실행→파일→옵션2.보안 센터→보안 센터 설정3.제한된 보기→‘인터넷에서 가...,cesco3_6.pdf,,cesco3,0.844935,2024-08-30,2024-08-30


In [41]:
query = "PDF 파일 열람 오류 해결방법 알려주세요"
df_context = run_vector_search_kr(query=query, metric="L2DISTANCE", k=5)
df_context

,ProblemDescription,ProblemCategory,ProblemKeyword,Solution,SolutionDoc,AdditionalInfo,Code,L2D_SIM,CreateDate,UpdateDate
0,PDF 파일 열람 오류,MS Office 및 기타 프로그램,PDF,1.PDF 파일 형식에 맞는 연결 프로그램을 지정하세요. 연결 프로그램을 사용자가 ...,cesco3_0.pdf,Adobe Acrobat Reader DC 설치 방법 참고Adobe Acroba...,cesco3,0.422898,2024-08-30,2024-08-30
1,"PPT(파워포인트, PowerPoint) 파일 열람 오류",MS Office 및 기타 프로그램,PPT,1.윈도우→설정2.‘앱’ 클릭3.‘기본 앱’ 클릭4.‘파일 형식별 기본 앱 선택’ ...,cesco3_15.pdf,,cesco3,0.498002,2024-08-30,2024-08-30
2,"그룹웨어 메일, 게시판에 첨부된 문서 '열기'클릭 시 열람 오류",그룹웨어,그룹웨어,"1.크롬, 엣지 사이트 접속2.키보드 Ctrl + Shift + Delete 클릭3...",cesco5_11.pdf,,cesco5,0.541984,2024-08-30,2024-08-30
3,'DRM RunDLL / f_piimgr.exe 시스템 오류 해결 방법,보안프로그램,문서보안(DRM),1.윈도우 검색 (윈도우 키 + S) → DRM 응급복구 → 관리자 권한으로 실행2...,cesco8_7.pdf,,cesco8,0.555076,2024-08-30,2024-08-30
4,그룹웨어 게시물에 첨부된 문서 실행 시 빈 화면으로 보이는 문서열람 오류,MS Office 및 기타 프로그램,MS Office,1.문서 실행→파일→옵션2.보안 센터→보안 센터 설정3.제한된 보기→‘인터넷에서 가...,cesco3_6.pdf,,cesco3,0.556894,2024-08-30,2024-08-30


In [49]:
# test_data = [
#     "SC 법인 차량에 에어컨이 안나옵니다.",
#     "차세대시스템 내 DB 구조가 궁금합니다.",
#     "DATA를 DB에 올리는 방법이 궁금합니다.",
#     "바퀴벌레 해충 동정 법이 궁금합니다.",
#     "시궁쥐와 들쥐의 차이점이 궁금합니다.",
#     "세스코 터치센터의 주소가 궁금해요.",
#     "세스카페의 아메리카노 가격이 궁금합니다.",
#     "세스코 터치센터 근처 버스정류장이 어디있나요",
#     "터치센터에서 강일역 가능 방법이 궁금해요",
#     "셔틀버스 탑승 시간을 알려주세요.",
#     "출근 할 때 잠실역에서 셔틀버스 어디서 타는지 알고 싶어요.",
#     "본사 직원 출근 시간이 궁금합니다.",
#     "IT 서비스 데스크 직통 번호를 알려주세요.",
#     "회사에서 지급한 노트북이 고장났어요.",
#     "윈도우 설치 방법이 궁금합니다.",
#     "세스톡 로그인이 안됩니다.",
#     "연차 신청 방법이 궁금합니다.",
#     "내 연차가 얼마나 남았는지 궁금해.",
#     "이번 달 월급 명세서 확인하려면 어디로 가야하나요",
#     "회사창립기념일이 언제인가요?",
#     "DLP 로그인이 안돼요"
# ]

In [4]:
test_data = [
    "SC 법인 차량에 에어컨이 안나옵니다.",
    "차세대시스템 내 DB 구조가 궁금합니다.",
    "DATA를 DB에 올리는 방법이 궁금합니다.",
    "바퀴벌레 해충 동정 법이 궁금합니다.",
    "시궁쥐와 들쥐의 차이점이 궁금합니다.",
    "세스코 터치센터의 주소가 궁금해요.",
    "세스카페의 아메리카노 가격이 궁금합니다.",
    "세스코 터치센터 근처 버스정류장이 어디있나요",
    "터치센터에서 강일역 가능 방법이 궁금해요",
    "셔틀버스 탑승 시간을 알려주세요.",
    "출근 할 때 잠실역에서 셔틀버스 어디서 타는지 알고 싶어요.",
    "본사 직원 출근 시간이 궁금합니다.",
    "IT 서비스 데스크 직통 번호를 알려주세요.",
    "회사에서 지급한 노트북이 고장났어요.",
    "윈도우 설치 방법이 궁금합니다.",
    "세스톡 로그인이 안됩니다.",
    "연차 신청 방법이 궁금합니다.",
    "내 연차가 얼마나 남았는지 궁금해.",
    "이번 달 월급 명세서 확인하려면 어디로 가야하나요",
    "회사창립기념일이 언제인가요?",
    "급여 명세서를 출력하고 싶어요",
    "본사에서 가장 가까운 역",
    "구내 식당 점심 메뉴 어디서 볼 수 있어?",
    "SC 신입 교육 일정 알려줘",
    "DLP 오류 해결하는 법 알려줘",
    "세스넷 접속 방법",
    "지출 결의서 작성법",
    "그룹웨어에서 문서 기안하는 법",
    "윈도우 정품 인증하는 법",
    "사무용품 신청하는 법",
    "총무팀 연락처 알려주세요",
    "차세대 그룹 웨어 사용법",
    "신입 사원 멘토링 내용 작성 어디다가 하나요",
    "퇴직금 정산 신청법",
    "정보전략팀 연락 방법",
    "데이터베이스 권한 신청하는 법",
    "ERP 관리자 계정 누구한테 문의해?",
    "세스코몰 직원 할인 받는법",
    "신입 사원 교육 시험 컷트라인 알려줘",
    "바퀴벌레가 사람한테 미치는 피해는?",
    "세스코 화이트 운영시간 알려줘",
    "휴가 신청 방법 알려줘",
    "내 휴가 얼마나 남았을까?",
    "노트북 반출 신청 방법",
    "세스코 창립 기념일은 언제야?",
    "차세대 시스템 로그인 하는 법 알려줘",
    "세스코 마스코트 누구야",
    "빈대 탐지견 이름 알려줘",
    "SAC 대시보드 제작 가이드 알려줘",
    "이번 달 월급 명세서 확인하려면 어디로 가야하나요",
    "pdf 안열릴 때 해결 방법",
    "한글 워드 파일이 안열려요",
    "알약이 작동이 안됩니다.",
    "ppt 파일이 열리지가 않네요",
    "사내 디스크에서 문서 파일 가져오고 싶어요",
    "법인 차량 최대 규정 속도를 알고 싶어요",
    "법인 차량 공회전 시간 얼마나 되나요",
    "유류비 신청 어디서 하면 되나요",
    "미팅에 필요한 다과비용 청구 가능한가요",
    "사내 업무용 도서 신청하고 싶습니다",
    "노트북이 고장났는데 누구한테 말해야 할까요",
    "모니터가 잘 안나와요 교체 되나요",
    "이번주 터치센터 점심 시간 알려주세요",
    "세스넷 권한 신청 누구한테 하면 될까요",
    "정보전략팀 상무님 성함이 궁금합니다",
    "세스코 연혁을 알 수 있을까요",
    "회장님 성함이 궁금합니다.",
    "보안 프로그램 다운 받는 가이드 필요합니다.",
    "인터넷 연결이 안되는데 해결해주세요",
    "사내 교육 인터넷 강의 어디서 보면 되요?",
    "모바일 앱 어디서 받으면 되나요",
    "SRS 로그인이 안됩니다.",
    "그룹웨어 비밀번호 찾는 법",
    "지사 요청 불가능 할 때 해결 방법",
    "설치 확인 등록 중 문제가 발생했습니다 해결 방법",
    "연장 근무 신청하는 방법",
    "연장 근무 신청 안했는데 근로시간 초과 알림 왔을 때",
    "세스톡 개인 정보 변경하는 법?",
    "세스톡 쪽지 확인 안되는 경우",
    "ehr 로그인시 계정이 잠겼다고 나와요",
    "근태 신청시 전자 결재 오류가 발생해서 진행되지 않습니다.",
    "응답 오류가 발생했습니다 해결 방법",
    "게시글 문서 열람 권한이 없어서 안열리는 경우",
    "반출 문서 보안 푸는 방법",
    "세스코몰 사이트 주소 알려줘",
    "세스코몰 로그인이 안될 때 어떻게 하지",
    "사내 보안 프로그램은 뭐 깔면 돼?",
    "세스넷 권한 신청 누구한테 하면 될까요",
    "PC ip 확인하는 방법",
    "본사 무선 네트워크 연결 방법",
    "본사에서 와이파이가 잡히지 않을 때 해결법",
    "세스닉 이용 방법",
    "퇴사 후 노트북 반납 방법",
    "세스코 워드 작성 가이드 알려줘",
    "세스코 글씨체 다운 받는 법",
    "세스코 본사 복장 규정 알려줘",
    "sc 고객 응대 가이드 알려주세요",
    "세스코 복지를 알려주세요",
    "세스코 직급 체계가 궁금합니다",
    "명함 신청하는 방법 알려주세요"
]

In [5]:
s_list = []

for i in range(len(test_data)):
    df_context = run_vector_search_kr(query=test_data[i], metric="L2DISTANCE", k=1)
    v_s = pd.Series(test_data[i], name='query')
    for j in range(1):
        context_series = df_context.iloc[j]
        combined_series = pd.concat([v_s, context_series])
        s_list.append(combined_series)

s_df = pd.DataFrame(s_list)

s_df.to_csv('output5.csv', index=False, encoding='utf-8-sig')

#유클리드 거리 계산법으로 테스트 진행했을 때 0.5의 값을 기준으로 한다

In [56]:
query = "PDF 파일 열람 오류 해결방법 알려주세요"
df_context = run_vector_search_kr(query=query, metric="L2DISTANCE", k=5)
df_context

,ProblemDescription,ProblemCategory,ProblemKeyword,Solution,SolutionDoc,AdditionalInfo,Code,L2D_SIM,CreateDate,UpdateDate
0,PDF 파일 열람 오류,MS Office 및 기타 프로그램,PDF,1.PDF 파일 형식에 맞는 연결 프로그램을 지정하세요. 연결 프로그램을 사용자가 ...,cesco3_0.pdf,Adobe Acrobat Reader DC 설치 방법 참고Adobe Acroba...,cesco3,0.422898,2024-08-30,2024-08-30
1,"PPT(파워포인트, PowerPoint) 파일 열람 오류",MS Office 및 기타 프로그램,PPT,1.윈도우→설정2.‘앱’ 클릭3.‘기본 앱’ 클릭4.‘파일 형식별 기본 앱 선택’ ...,cesco3_15.pdf,,cesco3,0.498002,2024-08-30,2024-08-30
2,"그룹웨어 메일, 게시판에 첨부된 문서 '열기'클릭 시 열람 오류",그룹웨어,그룹웨어,"1.크롬, 엣지 사이트 접속2.키보드 Ctrl + Shift + Delete 클릭3...",cesco5_11.pdf,,cesco5,0.541984,2024-08-30,2024-08-30
3,'DRM RunDLL / f_piimgr.exe 시스템 오류 해결 방법,보안프로그램,문서보안(DRM),1.윈도우 검색 (윈도우 키 + S) → DRM 응급복구 → 관리자 권한으로 실행2...,cesco8_7.pdf,,cesco8,0.555076,2024-08-30,2024-08-30
4,그룹웨어 게시물에 첨부된 문서 실행 시 빈 화면으로 보이는 문서열람 오류,MS Office 및 기타 프로그램,MS Office,1.문서 실행→파일→옵션2.보안 센터→보안 센터 설정3.제한된 보기→‘인터넷에서 가...,cesco3_6.pdf,,cesco3,0.556894,2024-08-30,2024-08-30


In [65]:
promptTemplate_fstring = """
You are a friendly and helpful AI assistant. 
Based on the provided context, please answer the user's question in a clear and polite manner, ensuring the response is easy to understand.
Context:
{context}
Question:.
{query}
Answer:
"""

In [7]:
promptTemplate_fstring = """
You are a friendly and helpful AI assistant.
Based on the provided context, please answer the user's question in a clear and polite manner, ensuring the response is easy to understand. 
Make sure to include all the content from the provided context and do not omit any details. 
If there is information under "etc", include that additional information in your answer as well.
Context:
{context}
Question:
{query}
Answer:
"""


In [16]:
from langchain_core.prompts import PromptTemplate
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client
proxy_client = get_proxy_client('gen-ai-hub')

promptTemplate = PromptTemplate.from_template(promptTemplate_fstring)

def ask_llm(query: str, k1_context: pd.Series) -> str:
    
    context = f"""category: {k1_context["ProblemCategory"]}, keyword: {k1_context["ProblemKeyword"]},content: {k1_context["ProblemDescription"]},{k1_context["Solution"]}, etc: {k1_context["AdditionalInfo"]}"""
    
    prompt = promptTemplate.format(query=query, context=context)
    
    print('\nAsking LLM...')

    llm = ChatOpenAI(deployment_id="d03974e89ef130ad", temperature=0)

    response = llm.invoke(prompt)

    print(response.content)

In [18]:
query = "SRS 로그인이 안됩니다"
df_context_5= run_vector_search_kr(query=query, metric="L2DISTANCE", k=5)
df_context_1 = df_context_5.iloc[0]
print(df_context_1)
response = ask_llm(query=query, k1_context=df_context_1)

ProblemDescription    사용자를 인증할 수 없습니다. 포털에 사용자 정보를 확인하세요.오류 발생 해결 방법
ProblemCategory                                          모바일서비스(SRS)
ProblemKeyword                                   모바일서비스(SRS), 로그인 오류
Solution                  ※설정 → 휴대전화 정보 → IMEI 번호 확인 후 포털에 사용자 등록 요청
SolutionDoc                                             cesco7_2.pdf
AdditionalInfo                                                      
Code                                                          cesco7
L2D_SIM                                                     0.449526
CreateDate                                                2024-08-30
UpdateDate                                                2024-08-30
Name: 0, dtype: object

Asking LLM...
SRS 로그인 오류가 발생했을 때는 사용자 인증에 문제가 있을 수 있습니다. 해결 방법으로는 다음과 같은 절차를 따라 해보시기 바랍니다:

1. 휴대전화의 '설정' 메뉴로 이동하세요.
2. '휴대전화 정보'를 선택하세요.
3. 'IMEI 번호'를 확인하세요.
4. 확인한 IMEI 번호를 가지고 포털에 사용자 등록을 요청하세요.

이 절차를 통해 사용자 인증 문제를 해결할 수 있습니다. 만약 문제가 계속된다면, 추가적인 도움을 위해 고객 지원 센터에 문의하시는 것도 좋은 방법입니다.
